In [10]:
import pandas as pd
df = pd.read_csv('./datasets/GB_data.csv') 

In [12]:
def truncate_outliers(df, col, lower_b=True, upper_b=True):
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    if lower_b == True:
        lower_bound = q1 - 1.5 * iqr
    if upper_b == True:
        upper_bound = q3 + 1.5 * iqr
    if lower_b == False:
        lower_bound = None
    if upper_b == False:
        upper_bound = None

    df[col] = df[col].clip(lower_bound, upper_bound)
    return df

In [14]:
df = truncate_outliers(df, 'GB_UKM_load_actual_entsoe_transparency', lower_b=True)

In [15]:
df.to_csv('./datasets/GB_data_new.csv', index=False)

In [16]:
import os
import pandas as pd
import subprocess
import shutil
import time
from utils.helper import extract_metrics_from_output, convert_results_into_df, running_time

# PatchTST 512

In [17]:
# Paths to files and data
data_path = os.getcwd() + "/datasets/"

script_path = "./PatchTST-main/PatchTST_supervised/run_longExp.py"

log_dir = f"logs/patchtst/"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

In [18]:
cuda_device = "3"
os.environ["CUDA_VISIBLE_DEVICES"] = cuda_device

# Dynamic variables
pred_lens = [24, 96, 168]
countries = ['GB']
num_cols = [5]
seq_lens = [512]

model = "PatchTST"
loss = "MSE"
itr=1

# Log file with all the results in 1 file
log_file_path = f"{log_dir}/{model}_no_revin.log"

# Parameters for tuning,but default
lr = 0.0001
n_heads = 16
e_layers = 3
d_model = 128
d_ff = 256
dropout = 0.2
batch_size = 128

# List to store the results
patchtst_results = []

In [25]:
start = time.time()

# Log file
with open(log_file_path, "w") as log_file:

    for i, country in enumerate(countries):
        
        country_start = time.time()
        statement_1 = f"\n=== Starting experiments for country: {country} ===\n"
        log_file.write(statement_1)
        print(statement_1)

        for pred_len in pred_lens:
            if country == 'DE' and pred_len == 24:
                seq_len=336
            else:
                seq_len = seq_lens[i]

            pred_len_start = time.time()
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"no_outliers_{country}_{seq_len}_{pred_len}_{country}"
            dataset = f"{country}_data_new.csv"
            
            # Arguments for the command
            command = f"""
            python {script_path} \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --pred_len {pred_len} \
              --e_layers {e_layers} \
              --factor 1 \
              --enc_in {num_cols[i]} \
              --c_out {num_cols[i]} \
              --des 'Exp' \
              --train_epochs 100 \
              --patience 10 \
              --n_heads {n_heads} \
              --d_model {d_model} \
              --d_ff {d_ff} \
              --dropout {dropout} \
              --fc_dropout {dropout} \
              --overlapping_windows \
              --scaler_type minmax \
              --if_relu \
              --loss_fnc {loss} \
              --itr {itr} --batch_size {batch_size} --learning_rate "{lr}"
            """

            # Log the country and prediction length
            log_file.write(f"\n--- Running model for {country}, pred_len={pred_len} ---\n")

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            process.wait()  # Wait for the process to finish
            #shutil.rmtree('./checkpoints' )  # delete checkpoint files

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr)

            # Log the extracted metrics and save them
            for iteration, scaled_metrics in enumerate(iteration_metrics, start=1):

                patchtst_results.append({
                    'Country': country,
                    'Pred_len': pred_len,
                    'Iteration': iteration,
                    'MSE': scaled_metrics[0],
                    'RMSE': scaled_metrics[1],
                    'MAE': scaled_metrics[2],
                    })
                
            pred_len_end = time.time()
            hours_int, mins_int, secs_int = running_time(pred_len_start, pred_len_end)
            statement_3 = "Intermediate time for {} and pred_len {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, pred_len, hours_int, mins_int, secs_int)
            log_file.write(statement_3)
            print(statement_3)

        country_end = time.time()
        hours_c, mins_c, secs_c = running_time(country_start, country_end)
        statement_4 = "Intermediate time for {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, hours_c, mins_c, secs_c)
        log_file.write(statement_4)
        print(statement_4)

    end = time.time()
    hours, mins, secs = running_time(start, end)
    statement_5 = "Total time: {:0>2}h:{:0>2}m:{:05.2f}s".format(hours, mins, secs)
    log_file.write(statement_5)
    print(statement_5)


=== Starting experiments for country: GB ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='no_outliers_GB_512_24_GB', model='PatchTST', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work-1/datasets/', data_path='GB_data_new.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='minmax', if_relu=True, channel_mixing=0, seq_len=512, label_len=48, pred_len=24, inverse=False, loss_fnc='MSE', fc_dropout=0.2, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=5, dec_in=7, c_out=5, d_model=128, n_heads=16, e_layers=3, d_layers=1, d_ff=256, moving_avg=25, factor=1, distil=True, dropout=0.2, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=1, train_epochs=100, batch_size=128, patience=10,

In [ ]:
path = 'results/patchtst'

patchtst_df = convert_results_into_df(patchtst_results, if_loss_fnc=False).round(4)
patchtst_df.to_csv(os.path.join(path, 'GB_patchtst_512.csv'))
patchtst_df.round(4)

MSE    RMSE     MAE
Country Pred_len                        
GB      24        0.0245  0.1567  0.1035
        96        0.0402  0.2005  0.1420
        168       0.0427  0.2067  0.1479

In [19]:
patchtst_results = []
start = time.time()

# Log file
with open(log_file_path, "w") as log_file:

    for i, country in enumerate(countries):
        
        country_start = time.time()
        statement_1 = f"\n=== Starting experiments for country: {country} ===\n"
        log_file.write(statement_1)
        print(statement_1)

        for pred_len in pred_lens:
            if country == 'DE' and pred_len == 24:
                seq_len=336
            else:
                seq_len = seq_lens[i]

            pred_len_start = time.time()
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"no_outliers_no_revin_{country}_{seq_len}_{pred_len}_{country}"
            dataset = f"{country}_data_new.csv"
            
            # Arguments for the command
            command = f"""
            python {script_path} \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --pred_len {pred_len} \
              --e_layers {e_layers} \
              --factor 1 \
              --enc_in {num_cols[i]} \
              --c_out {num_cols[i]} \
              --des 'Exp' \
              --train_epochs 100 \
              --patience 10 \
              --n_heads {n_heads} \
              --d_model {d_model} \
              --d_ff {d_ff} \
              --dropout {dropout} \
              --fc_dropout {dropout} \
              --overlapping_windows \
              --scaler_type minmax \
              --if_relu \
              --loss_fnc {loss} \
              --revin 0 \
              --itr {itr} --batch_size {batch_size} --learning_rate "{lr}"
            """

            # Log the country and prediction length
            log_file.write(f"\n--- Running model for {country}, pred_len={pred_len} ---\n")

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            process.wait()  # Wait for the process to finish
            #shutil.rmtree('./checkpoints' )  # delete checkpoint files

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr)

            # Log the extracted metrics and save them
            for iteration, scaled_metrics in enumerate(iteration_metrics, start=1):

                patchtst_results.append({
                    'Country': country,
                    'Pred_len': pred_len,
                    'Iteration': iteration,
                    'MSE': scaled_metrics[0],
                    'RMSE': scaled_metrics[1],
                    'MAE': scaled_metrics[2],
                    })
                
            pred_len_end = time.time()
            hours_int, mins_int, secs_int = running_time(pred_len_start, pred_len_end)
            statement_3 = "Intermediate time for {} and pred_len {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, pred_len, hours_int, mins_int, secs_int)
            log_file.write(statement_3)
            print(statement_3)

        country_end = time.time()
        hours_c, mins_c, secs_c = running_time(country_start, country_end)
        statement_4 = "Intermediate time for {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, hours_c, mins_c, secs_c)
        log_file.write(statement_4)
        print(statement_4)

    end = time.time()
    hours, mins, secs = running_time(start, end)
    statement_5 = "Total time: {:0>2}h:{:0>2}m:{:05.2f}s".format(hours, mins, secs)
    log_file.write(statement_5)
    print(statement_5)


=== Starting experiments for country: GB ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='no_outliers_no_revin_GB_512_24_GB', model='PatchTST', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work-1/datasets/', data_path='GB_data_new.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='minmax', if_relu=True, channel_mixing=0, seq_len=512, label_len=48, pred_len=24, inverse=False, loss_fnc='MSE', fc_dropout=0.2, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=0, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=5, dec_in=7, c_out=5, d_model=128, n_heads=16, e_layers=3, d_layers=1, d_ff=256, moving_avg=25, factor=1, distil=True, dropout=0.2, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=1, train_epochs=100, batch_size=128, pat

In [20]:
convert_results_into_df(patchtst_results, if_loss_fnc=False, itr=1)

Iteration       MSE      RMSE       MAE
Country Pred_len                                         
GB      24                1  0.026630  0.163186  0.109980
        96                1  0.042054  0.205070  0.146627
        168               1  0.043597  0.208798  0.150366

# Informer

In [28]:
# Paths to files and data
data_path = os.getcwd() + "/datasets/"

script_path = "./PatchTST-main/PatchTST_supervised/run_longExp.py"

log_dir = f"logs/informer/"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

In [29]:
# Dynamic variables
seq_len = 96
model = "Informer"
itr = 1

# Log file with all the results in 1 file
log_file_path = f"{log_dir}/{model}_gb_no_outliers.log"

# Parameters for tuning
lr = 0.0001
#n_heads = 16
e_layers = 2
d_layers = 1
loss = "MSE"

os.environ["CUDA_VISIBLE_DEVICES"] = cuda_device

# Lists to store the results
informer_results = []

In [30]:
start = time.time()

# Log file
with open(log_file_path, "w") as log_file:

    for i, country in enumerate(countries):
        
        country_start = time.time()
        statement_1 = f"\n=== Starting experiments for country: {country} ===\n"
        log_file.write(statement_1)
        print(statement_1)

        for pred_len in pred_lens:

            pred_len_start = time.time()
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_{country}"
            dataset = f"{country}_data_new.csv"
            
            # Arguments for the command
            command = f"""
            python {script_path} \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --label_len 48 \
              --pred_len {pred_len} \
              --e_layers {e_layers} \
              --d_layers {d_layers} \
              --factor 5 \
              --enc_in {num_cols[i]} \
              --dec_in {num_cols[i]} \
              --c_out {num_cols[i]} \
              --dropout 0.1 \
              --des 'Exp' \
              --train_epochs 20 \
              --patience 5 \
              --overlapping_windows \
              --loss_fnc "{loss}" \
              --scaler_type minmax \
              --if_relu \
              --itr {itr} --batch_size 32 --learning_rate "{lr}"
            """

            # Log the country and prediction length
            log_file.write(f"\n--- Running model for {country}, pred_len={pred_len} ---\n")

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            process.wait()  # Wait for the process to finish
            #shutil.rmtree('./checkpoints' )  # delete checkpoint files

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr)

            # Log the extracted metrics and save them
            for iteration, scaled_metrics in enumerate(iteration_metrics, start=1):

                informer_results.append({
                    'Country': country,
                    'Pred_len': pred_len,
                    'Iteration': iteration,
                    'MSE': scaled_metrics[0],
                    'RMSE': scaled_metrics[1],
                    'MAE': scaled_metrics[2],
                    })
                
            pred_len_end = time.time()
            hours_int, mins_int, secs_int = running_time(pred_len_start, pred_len_end)
            statement_3 = "Intermediate time for {} and pred_len {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, pred_len, hours_int, mins_int, secs_int)
            log_file.write(statement_3)
            print(statement_3)

        country_end = time.time()
        hours_c, mins_c, secs_c = running_time(country_start, country_end)
        statement_4 = "Intermediate time for {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, hours_c, mins_c, secs_c)
        log_file.write(statement_4)
        print(statement_4)

    end = time.time()
    hours, mins, secs = running_time(start, end)
    statement_5 = "Total time: {:0>2}h:{:0>2}m:{:05.2f}s".format(hours, mins, secs)
    log_file.write(statement_5)
    print(statement_5)


=== Starting experiments for country: GB ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='GB_96_24_GB', model='Informer', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work-1/datasets/', data_path='GB_data_new.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='minmax', if_relu=True, channel_mixing=0, seq_len=96, label_len=48, pred_len=24, inverse=False, loss_fnc='MSE', fc_dropout=0.05, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=5, dec_in=5, c_out=5, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=5, distil=True, dropout=0.1, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=1, train_epochs=20, batch_size=32, patience=5, learning_rate=0

In [33]:
informer_df = convert_results_into_df(informer_results, if_loss_fnc=False, itr=1)

In [35]:
path = 'results/informer'

informer_df.to_csv(os.path.join(path, 'GB_informer.csv'))
informer_df.round(4)

Iteration     MSE    RMSE     MAE
Country Pred_len                                   
GB      24                1  0.0313  0.1769  0.1219
        96                1  0.0499  0.2233  0.1605
        168               1  0.0635  0.2520  0.1772